In [1]:
import gym
import ray

In [2]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [3]:
# Initialize Ray
ray.shutdown()
ray.init(ignore_reinit_error=True,num_cpus=8, num_gpus=1)

2020-09-27 16:49:55,282	INFO resource_spec.py:231 -- Starting Ray with 13.57 GiB memory available for workers and up to 6.81 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-09-27 16:49:55,807	INFO services.py:1193 -- View the Ray dashboard at localhost:8266


{'node_ip_address': '192.168.7.73',
 'raylet_ip_address': '192.168.7.73',
 'redis_address': '192.168.7.73:25146',
 'object_store_address': '/tmp/ray/session_2020-09-27_16-49-55_281306_45839/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-09-27_16-49-55_281306_45839/sockets/raylet',
 'webui_url': 'localhost:8266',
 'session_dir': '/tmp/ray/session_2020-09-27_16-49-55_281306_45839'}

In [4]:
# https://docs.ray.io/en/latest/tune/user-guide.html#parallelism-gpus
ray.cluster_resources()

{'GPU': 1.0,
 'node:192.168.7.73': 1.0,
 'memory': 278.0,
 'GPUType:GTX': 1.0,
 'object_store_memory': 96.0,
 'CPU': 8.0}

In [5]:
print("Dashboard URL: http://{}".format(ray.get_webui_url()))

Dashboard URL: http://localhost:8266


### Register MineRL Gym Environment to RLlib

In [6]:
from minerl_rllib.envs import register

/home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
register()

In [8]:
from ray import tune
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples

import ray.rllib.agents.ppo as ppo
from ray.rllib.agents.ppo import PPOTrainer
# https://docs.ray.io/en/latest/rllib-training.html#tuned-examples
# config = ppo.DEFAULT_CONFIG.copy()
# config["num_gpus"] = 1
# config["num_workers"] = 1
# config["num_envs_per_worker"] = 1
# config["eager"] = False
# trainer = ppo.PPOTrainer(config=config, env="MineRLNavigateDenseVectorObf-v0")


tune.run(PPOTrainer,
         config={"env": "MineRLNavigateDenseVectorObf-v0",
                 "num_gpus": 1,
                 "num_workers": 1,
                 "num_envs_per_worker": 1,
                 "eager": False,
                 "use_pytorch": True,
#                  'monitor':True, 
#                  "model": {
#                     "custom_model": "fc_pov",
#                     }
                 }
         ,stop={"training_iteration": 3})  

Instructions for updating:
non-resource variables are not supported in the long term


Trial name,status,loc
PPO_MineRLNavigateDenseVectorObf-v0_27406_00000,RUNNING,


(pid=45922) WARNING:tensorflow:From /home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=45922) Instructions for updating:
(pid=45922) non-resource variables are not supported in the long term
(pid=45922) /home/blackbox/anaconda3/envs/minerl/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
(pid=45922)   warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
(pid=45922) 2020-09-27 16:50:02,083	WARNING deprecation.py:30 -- DeprecationWarning: `use_pytorch` has been deprecated. Use `framework=torch` instead. This will raise an error in the future!
(pid=45922) 2020-09-27 16:50:02,083	WARNING deprecation.py:30 -- DeprecationWarning: `eager` has been deprecated. Use `framework=tfe` instead. This will raise an error in the future!


Result for PPO_MineRLNavigateDenseVectorObf-v0_27406_00000:
  custom_metrics: {}
  date: 2020-09-27_16-51-49
  done: false
  episode_len_mean: .nan
  episode_reward_max: .nan
  episode_reward_mean: .nan
  episode_reward_min: .nan
  episodes_this_iter: 0
  episodes_total: 0
  experiment_id: 5bfaa4612e8f4cfab30958732c510cf7
  experiment_tag: '0'
  hostname: blackbox
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.2
        cur_lr: 5.0e-05
        entropy: 90.85596132278442
        entropy_coeff: 0.0
        kl: 0.024287886277306825
        policy_loss: -0.04235954786418006
        total_loss: 0.16618813131935894
        vf_explained_var: 0.0012483876198530197
        vf_loss: 0.20369009929709136
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1
  node_ip: 192.168.7.73
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 36.96266666666667
    ram_util_percent: 50.932666666666

Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_27406_00000,RUNNING,192.168.7.73:45922,1,105.137,4000,nan


...

Result for PPO_MineRLNavigateDenseVectorObf-v0_27406_00000:
  custom_metrics: {}
  date: 2020-09-27_16-53-01
  done: false
  episode_len_mean: 6000.0
  episode_reward_max: 1.7386474609375
  episode_reward_mean: 1.7386474609375
  episode_reward_min: 1.7386474609375
  episodes_this_iter: 1
  episodes_total: 1
  experiment_id: 5bfaa4612e8f4cfab30958732c510cf7
  experiment_tag: '0'
  hostname: blackbox
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.30000000000000004
        cur_lr: 5.0e-05
        entropy: 90.74326992034912
        entropy_coeff: 0.0
        kl: 0.02322320465464145
        policy_loss: -0.022967456898186356
        total_loss: 0.08791951002785936
        vf_explained_var: -2.1070241928100586e-05
        vf_loss: 0.10392000596038997
    num_steps_sampled: 8000
    num_steps_trained: 8000
  iterations_since_restore: 2
  node_ip: 192.168.7.73
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 31

Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_27406_00000,RUNNING,192.168.7.73:45922,2,176.955,8000,1.73865


Result for PPO_MineRLNavigateDenseVectorObf-v0_27406_00000:
  custom_metrics: {}
  date: 2020-09-27_16-54-09
  done: true
  episode_len_mean: 6000.0
  episode_reward_max: 1.7386474609375
  episode_reward_mean: 1.1825065612792969
  episode_reward_min: 0.6263656616210938
  episodes_this_iter: 1
  episodes_total: 2
  experiment_id: 5bfaa4612e8f4cfab30958732c510cf7
  experiment_tag: '0'
  hostname: blackbox
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.45000000000000007
        cur_lr: 5.0e-05
        entropy: 90.64632725715637
        entropy_coeff: 0.0
        kl: 0.014618130109738559
        policy_loss: -0.02481837710365653
        total_loss: 0.13858625059947371
        vf_explained_var: -0.00012756511569023132
        vf_loss: 0.15682647447101772
    num_steps_sampled: 12000
    num_steps_trained: 12000
  iterations_since_restore: 3
  node_ip: 192.168.7.73
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_perc

Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_27406_00000,TERMINATED,,3,244.911,12000,1.18251


Trial name,status,loc,iter,total time (s),ts,reward
PPO_MineRLNavigateDenseVectorObf-v0_27406_00000,TERMINATED,,3,244.911,12000,1.18251
